In [2]:
pip install torch torchvision pillow gradio

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
    --------------------------------------- 2.6/204.1 MB 13.7 MB/s eta 0:00:15
    --------------------------------------- 5.0/204.1 MB 12.1 MB/s eta 0:00:17
   - -------------------------------------- 7.3/204.1 MB 11.6 MB/s eta 0:00:17
   - -------------------------------------- 9.4/204.1 MB 11.5 MB/s eta 0:00:17
   -- ------------------------------------- 12.1/204.1 MB 11.1 MB/s eta 0:00:18
   -- ------------------------------------- 13.6/204.1 MB 10.6 MB/s eta 0:00:19
   -- ------------------------------------- 14.9/204.1 MB 10.2 MB/s eta 0:00:19
   --- ------------------------------------ 16.0/204.1 MB 9.4 MB/s eta 0:00:20
   --- ------------------------------------ 17.3/204.1 MB 8.9 MB/s eta 0:

In [2]:
pip install jupyter streamlit

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import io
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np

# Load the pre-trained model and modify the final layer
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # Assuming 7 categories

# Load model weights
try:
    model.load_state_dict(torch.load('trashbox_model.pth', map_location=torch.device('cpu')))
    model.eval()
except Exception as e:
    st.error(f"Error loading model: {e}")

# Waste categories
categories = ["Cardboard", "E-Waste", "Glass", "Medical", "Metal", "Paper", "Plastic"]

# Data storage for dashboard
if "waste_data" not in st.session_state:
    st.session_state.waste_data = {category: 0 for category in categories}

# Variable to keep track of the last classified category
if "last_category" not in st.session_state:
    st.session_state.last_category = ""

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Classification function
def classify_images(uploaded_files):
    results = []
    for uploaded_file in uploaded_files:
        try:
            image = Image.open(uploaded_file)
            img_tensor = transform(image).unsqueeze(0)

            with torch.no_grad():
                outputs = model(img_tensor)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)[0].numpy()
                predicted_index = np.argmax(probabilities)
                predicted_class = categories[predicted_index]
                confidence = probabilities[predicted_index] * 100
                st.session_state.waste_data[predicted_class] += 1  # Track waste collection
                st.session_state.last_category = predicted_class  # Update last category

            results.append(f"{predicted_class} (Confidence: {confidence:.2f}%)")
        except Exception as e:
            results.append(f"Error: {str(e)}")
    
    return "\n".join(results)

# Visualization function for waste data
def generate_dashboard():
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(st.session_state.waste_data.keys(), st.session_state.waste_data.values(), color='green')
    ax.set_xlabel("Waste Category")
    ax.set_ylabel("Quantity Collected")
    ax.set_title("Waste Collection Statistics")
    ax.set_xticklabels(st.session_state.waste_data.keys(), rotation=45)
    
    return fig

# Recycling instructions
def get_recycling_instructions():
    recycling_instructions = {
        "Cardboard": "Flatten and place in the recycling bin. Ensure it's clean and dry.",
        "E-Waste": "Take to designated e-waste collection centers or drop-off points.",
        "Glass": "Rinse and place in the glass recycling bin. Separate colors if needed.",
        "Medical": "Dispose of according to local medical waste regulations.",
        "Metal": "Rinse and recycle. Most metals are recyclable.",
        "Paper": "Ensure it's clean and dry before recycling.",
        "Plastic": "Check resin codes and recycle accordingly."
    }
    return recycling_instructions.get(st.session_state.last_category, "No instructions available.")

# Streamlit UI
st.title("♻️ AI-Powered Waste Classifier")
st.write("Upload waste images to classify and view recycling instructions.")

# Upload images
uploaded_files = st.file_uploader("Choose images...", type=["jpg", "png", "jpeg"], accept_multiple_files=True)

if uploaded_files:
    # Display uploaded images
    st.image([Image.open(file) for file in uploaded_files], caption="Uploaded Images", use_column_width=True)

    # Perform classification
    classification_results = classify_images(uploaded_files)
    st.subheader("Classification Results")
    st.text(classification_results)

    # Display waste collection dashboard
    st.subheader("Waste Collection Dashboard")
    st.pyplot(generate_dashboard())

# Recycling instructions button
if st.button("Get Recycling Instructions"):
    st.subheader("Recycling Instructions")
    st.write(get_recycling_instructions())


2025-01-31 13:52:30.823 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-01-31 13:52:31.148 
  command:

    streamlit run C:\Users\ksbas\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
